In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

In [111]:
train= pd.read_csv('free-ai-classes-in-every-city-hackathon-2023\Housing_dataset_train.csv')
test=pd.read_csv('free-ai-classes-in-every-city-hackathon-2023\Housing_dataset_test.csv')

In [112]:
train.shape, test.shape

((14000, 7), (6000, 6))

In [113]:
train.head()

,ID,loc,title,bedroom,bathroom,parking_space,price
0,3583,Katsina,Semi-detached duplex,2.0,2.0,1.0,1149999.565
1,2748,Ondo,Apartment,NaN,2.0,4.0,1672416.689
2,9261,Ekiti,NaN,7.0,5.0,NaN,3364799.814
3,2224,Anambra,Detached duplex,5.0,2.0,4.0,2410306.756
4,10300,Kogi,Terrace duplex,NaN,5.0,6.0,2600700.898


In [114]:
#Check for the percetage of missing values
train.isnull().sum() * 100 / len(train)

ID                0.000000
loc              12.950000
title            12.300000
bedroom          12.850000
bathroom         12.892857
parking_space    12.935714
price             0.000000
dtype: float64

In [115]:
def cleandf(train):
    train = train.dropna(subset=['loc'])
    train.reset_index(drop=True, inplace=True)
    # fill the missing value in title with the most occuring value
    train['title'] = train['title'].fillna(train['title'].mode()[0])
    train['bedroom'] = train['bedroom'].fillna(train['bedroom'].mode()[0])
    train['parking_space'] = train['parking_space'].fillna(train['parking_space'].mode()[0])
    train['bathroom'] = train['bathroom'].fillna(train['bathroom'].mean())
    
    return train

In [117]:
train = cleandf(train)

C:\Users\Techa\AppData\Local\Temp\ipykernel_5364\2703087038.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['title'] = train['title'].fillna(train['title'].mode()[0])
C:\Users\Techa\AppData\Local\Temp\ipykernel_5364\2703087038.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['bedroom'] = train['bedroom'].fillna(train['bedroom'].mode()[0])
C:\Users\Techa\AppData\Local\Temp\ipykernel_5364\2703087038.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [118]:
def tranfm(train):
    oe= OrdinalEncoder()
    ohe= OneHotEncoder()
    train['loc'] = oe.fit_transform(train[['loc']])

    one_hot_encoded = ohe.fit_transform(train[['title']])

    # Convert the sparse matrix to a pandas DataFrame
    encoded_df = pd.DataFrame(one_hot_encoded.toarray(), columns=ohe.get_feature_names_out(['title']))

    # Concatenate the encoded DataFrame with the original DataFrame
    train = pd.concat([train, encoded_df], axis=1)
    return train

In [119]:
train = tranfm(train)

In [120]:
train.drop(['ID'], axis=1, inplace=True)
train.drop(['title'], axis=1, inplace=True)
y=train.pop('price')
X=train.values

In [121]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=123)
xtrain.shape, xtest.shape, ytrain.shape, ytest.shape

((8530, 14), (3657, 14), (8530,), (3657,))

In [130]:
rf = RandomForestRegressor()
lr = LinearRegression()

In [127]:
def model_b(model, xtrain, ytrain, xtest, ytest):
    model.fit(xtrain, ytrain)
    acc = model.score(xtest, ytest)
    pred = model.predict(xtest)
    return acc, pred 

In [137]:
lracc, lrpred = model_b(lr, xtrain, ytrain, xtest, ytest)
print("Logistic regression is ", lracc)

rfacc, rfpred = model_b(rf, xtrain, ytrain, xtest, ytest)
print("RandomForest regression is ", rfacc)

Logistic regression is  0.5379904204835804
RandomForest regression is  0.6935534204034662


In [132]:
lracc

0.5379904204835804

#--------------------------------

In [76]:
# xtrain['loc'] = oe.fit_transform(xtrain[['loc']])
# xtest['loc'] = oe.transform(xtest[['loc']])





# one_hot_encoded = ohe.fit_transform(xtrain[['title']])
# ohe_coded_test = ohe.transform(xtest[['title']])

# # Convert the sparse matrix to a pandas DataFrame
# encoded_df = pd.DataFrame(one_hot_encoded.toarray(), columns=ohe.get_feature_names_out(['title']))
# encoded_df_test = pd.DataFrame(ohe_coded_test.toarray(), columns=ohe.get_feature_names_out(['title']))

# # Concatenate the encoded DataFrame with the original DataFrame
# xtrain = pd.concat([xtrain, encoded_df], axis=1)
# xtest = pd.concat([xtest, encoded_df_test], axis=1)

# xtrain.drop('title', axis=1, inplace=True)
# xtest.drop('title', axis=1, inplace=True)


#-------------------

In [133]:
test.head()

,ID,loc,title,bedroom,bathroom,parking_space
0,845,Kano,Penthouse,4,1,2
1,1924,Adamawa,Apartment,2,2,4
2,10718,Adamawa,Bungalow,2,7,2
3,12076,Lagos,Mansion,9,5,2
4,12254,Gombe,Semi-detached duplex,5,6,1


In [134]:
df=test.copy()